In [2]:
#def nunpia_crawling(search_name) , 전체 라인 한칸씩 탭하고 input()제거할 것.
import bs4
import requests
from urllib import parse

print("작품이름 -> ")
search_name=input()
tmp = search_name
search_name_nonspace=search_name.replace(" ","")

url = "https://novel.munpia.com/page/hd.platinum/view/search/keyword/" + parse.quote(tmp) + "/order/search_result"


r=requests.get(url)
c=r.content
html = bs4.BeautifulSoup(c, "html.parser")

article=html.find('div',{'class','article'})
dl=article.find('dl',{'class','detail'})
href=dl.find('a').attrs['href']


url = href

r=requests.get(url)
c=r.content
html = bs4.BeautifulSoup(c, "html.parser")

print("\n")

intro=html.find('p',{'class','story'}).text
print("작품소개\n" + "----------------------------------------\n" + intro + "\n----------------------------------------")

cover_box=html.find('div',{'class','cover-box'})
img_src=cover_box.find('img').attrs['src']
print("이미지 : " + img_src)

box=html.find('div',{'class','dd detail-box'})
h2=box.find('h2')
exclusive=h2.find('span')
if(exclusive==None):
    print("독점여부 : 미독점")
    title=h2.text.replace('\n',"")
    print("제목 : " + title)
    #print("제목 : " + title,end='')
else:
    exclusive=exclusive.text
    print("독점여부 : " + exclusive)
    title=h2.text.split("독점\n")[1]
    print("제목 : " + title,end='')
    
p=box.find('p',{'class','meta-path'})
genre=p.find('strong').text
if ',' in genre:
    print("장르 : " + genre.split(',')[0] + ", " + genre.split(',')[1] )
else:
    print("장르 : " + genre)

fr=box.find('div',{'class','fr'})
prefer=fr.find('b').text
print("선호수 : " + prefer)

days = dict.fromkeys(["mon", "tue", "wed", "thu", "fri", "sat", "sun"], False)
novel_period=box.find('div',{'class','novel-period'})
period=""
if novel_period!=None:
    if(novel_period.find('span',{'class','pluszone-mon'})):
        days.update(dict.fromkeys(["mon"], True))
        period=period+"월,"
    if(novel_period.find('span',{'class','pluszone-tue'})):
        days.update(dict.fromkeys(["tue"], True))
        period=period+"화,"
    if(novel_period.find('span',{'class','pluszone-wed'})):
        days.update(dict.fromkeys(["wed"], True))
        period=period+"수,"
    if(novel_period.find('span',{'class','pluszone-thu'})):
        days.update(dict.fromkeys(["thu"], True))
        period=period+"목,"
    if(novel_period.find('span',{'class','pluszone-fri'})):
        days.update(dict.fromkeys(["fri"], True))
        period=period+"금,"
    if(novel_period.find('span',{'class','pluszone-sat'})):
        days.update(dict.fromkeys(["sat"], True))
        period=period+"토,"
    if(novel_period.find('span',{'class','pluszone-sun'})):
        days.update(dict.fromkeys(["sun"], True))
        period=period+"일,"
    
    
    if(all(value==True for value in days.values())==True):
        period="매일 "
    
    print("연재 주기 : " + period[:-1] + " 연재")

   

else:
    print("연재 주기 : 비주기적 연재")

    
age=box.find({'class','xui-icon xui-adult'})
if(age==None):
    print("연령등급 : 전체이용가")
else:
    age=age.text
    print("연령등급 : " + age)
    

iconset=box.find('p',{'class','iconset'})
complete=iconset.find('span',{'class','xui-icon xui-finish'})
if complete!=None:
    complete=complete.text
    print("완결여부 : " + complete)
else:
    print("완결여부 : 미완결")

writer_dl=box.find('dl',{'class','meta-author meta'})
author=writer_dl.find('strong').text
print("작가 : " + author)

etc_dl=box.findAll('dl',{'class','meta-etc meta'})
days_dl=etc_dl[0]
days_dd=days_dl.findAll('dd')
pub_date=days_dd[0].text
print("작품등록일 : " + pub_date)
#recently_update=days_dd[1].text
#print("최근 업데이트일 : " + recently_update)

number_dl=etc_dl[1]
number_dd=number_dl.findAll('dd')

total=number_dd[0].text
print("총 연재수 : " + total)

publisher="정보 없음"
print("출판사: " + publisher)
#recommend=number_dd[1].text
#print("추천수 : " + recommend)
#enjoyer=number_dd[2].text
#print("조회수 : " + enjoyer)

작품이름 -> 
1983


작품소개
----------------------------------------
만화광인 이윤환.
느닷없이 1983년, 자신과 같은 이름, 같은 모습의 인간으로 빙의되어 버렸다.

“오래된 만화를 좋아하긴 해도, 1980년대로 보내버리면 나더러 어쩌라는 거야?”

80년대에서 만화덕후의 삶이 시작된다.
----------------------------------------
이미지 : //cdn1.munpia.com/files/attach/2018/1227/004/NT1qYG4RErsKnjnv.jpg
독점여부 : 선독점
제목 : 1983 전생 만화왕
장르 : 현대판타지
선호수 : 13,509
연재 주기 : 비주기적 연재
연령등급 : 전체이용가
완결여부 : 완결
작가 : 장성필
작품등록일 : 2019.01.18 17:00
총 연재수 : 425 회
출판사: 정보 없음
